In [397]:
import numpy as np
from datetime import date
import pandas as pd
import altair as alt
import altair_latimes as lat

In [398]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Import

In [399]:
state_df = pd.read_csv("../../data/raw/snow/statewide-timeseries.csv", parse_dates=["date"])

In [400]:
normal_df = pd.read_csv("../../data/processed/snow/normal-swc.csv")

### Clean

Delete leap year day

In [401]:
normal_df = normal_df[
    ~((normal_df.month == 2) & (normal_df.day == 29))
]

In [402]:
normal_df["month"] = normal_df["month"].astype(str).str.zfill(2)
normal_df["day"] = normal_df["day"].astype(str).str.zfill(2)

In [403]:
normal_df["month-day"] = normal_df["month"] + "-" + normal_df["day"].astype(str)

In [404]:
state_df["month-day"] = state_df.date.dt.month.astype(str).str.zfill(2) + "-" + state_df.date.dt.day.astype(str).str.zfill(2)

### Get current water year

In [405]:
def getWaterYear(date):
    month = date.month
    day = date.day

    if month >= 10:
        water_year = date.year + 1
    else:
        water_year = date.year

    return water_year

In [406]:
today = date.today()

In [407]:
current_water_year = getWaterYear(today)
current_water_year

2023

### Merge to a series with the entire year

In [408]:
wy_start = str(current_water_year - 1)

In [409]:
start = pd.to_datetime( wy_start + "-10-01")

In [410]:
end = pd.to_datetime( str(current_water_year) + "-09-30")

In [411]:
print(start,"to", end)

2022-10-01 00:00:00 to 2023-09-30 00:00:00


In [412]:
dates = pd.Series(pd.date_range(start, end, freq="D"))
days = dates.diff().astype("timedelta64[D]").fillna(1).cumsum()
wy_df = pd.DataFrame({"year": dates.dt.year, "month": dates.dt.month, "month-day": dates.dt.month.astype(str).str.zfill(2) + "-" + dates.dt.day.astype(str).str.zfill(2)})
wy_df = wy_df.set_index(dates)
wy_df = wy_df.reset_index().rename(columns={"index":"date"})
wy_df

,date,year,month,month-day
0,2022-10-01,2022,10,10-01
1,2022-10-02,2022,10,10-02
2,2022-10-03,2022,10,10-03
3,2022-10-04,2022,10,10-04
4,2022-10-05,2022,10,10-05
...,...,...,...,...
360,2023-09-26,2023,9,09-26
361,2023-09-27,2023,9,09-27
362,2023-09-28,2023,9,09-28
363,2023-09-29,2023,9,09-29


In [413]:
merge = pd.merge(
    wy_df,
    state_df,
    how="left",
    on=["date","month-day"]
)

In [414]:
merge_normals = pd.merge(
    merge,
    normal_df.drop(columns=["month","day"], axis=1),
    how="left",
    on=["month-day"]
)

In [415]:
merge_normals.head()

,date,year,month,month-day,station_id,snwcavg,pctnorm,pctapr1,avg,wmonth,norm,avgAvgSwc
0,2022-10-01,2022,10,10-01,NaN,NaN,NaN,NaN,0.000,1,0.00,-0.01
1,2022-10-02,2022,10,10-02,NaN,NaN,NaN,NaN,0.000,1,0.03,0.00
2,2022-10-03,2022,10,10-03,NaN,NaN,NaN,NaN,0.000,1,0.00,-0.01
3,2022-10-04,2022,10,10-04,NaN,NaN,NaN,NaN,0.000,1,0.00,-0.01
4,2022-10-05,2022,10,10-05,NaN,NaN,NaN,NaN,0.001,1,0.00,0.00


In [416]:
#(merge_normals.loc[ (merge_normals['month-day'] == "1-20") ]['avg_pct_apr1'].sum()+merge_normals.loc[ (merge_normals['month-day'] == "1-22") ]['avg_pct_apr1'].sum())/2

In [417]:
merge_normals["pctapr1"] = merge_normals["pctapr1"] * 100
merge_normals["pctnorm"] = merge_normals["pctnorm"] * 100

In [418]:
merge_normals["avg"] = merge_normals["avg"] * 100

In [419]:
melt = pd.melt(
    merge_normals,
    id_vars="date",
    value_vars=["pctapr1","avg"]
)

In [420]:
alt.Chart(melt[melt.date >= "2022-10-01"]).mark_line().encode(
    x="date",
    y="value",
    color="variable",
    tooltip=["date","value"]
).properties(title="Snowpack as percentage of peak",width=600)

alt.Chart(...)

### Trim and rename columns

In [421]:
trim_df = merge_normals[[
    "date",
    "snwcavg",
    "pctnorm",
    "pctapr1",
    "avg",
    "avgAvgSwc"
]]

In [422]:
rename_df = trim_df.rename(columns={
    "snwcavg":"swc_avg",
    "pctnorm":"pct_normal",
    "pctapr1":"pct_apr_1",
    "avg": "apr_1_baseline_pct",
    "avgAvgSwc": "apr_1_baseline_value"
})

In [423]:
rename_df.to_csv("../../data/processed/snow/snow-water-equivalent.csv", index=False)